In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Префиксный код (импорты, вспомогательные функции/классы/гиперпараметры)

In [ ]:
# Imports
import numpy as np
import pandas as pd
import haversine as hs
import random

# Imports from
from haversine import Unit
from sklearn.model_selection import GroupKFold
from fuzzywuzzy import fuzz
from xgboost import XGBClassifier

In [ ]:
# Fix all random seeds
random.seed(42)

In [ ]:
# Custom Func's/Class's

def jaccard(list_a: list, list_b: list) -> float:
    a = set(list_a)
    b = set(list_b)
    return len(a & b) / len(a | b)


def jaccard_score(df_sub_a: pd.DataFrame, df_sub_b: pd.DataFrame) -> float:
    
    assert len(df_sub_a) == len(df_sub_b)
    df = pd.merge(df_sub_a, df_sub_b, on='id', suffixes=['_a','_b'])
    return (df.apply(lambda x: jaccard(x.matches_a.split(), x.matches_b.split()),
                     axis=1)).mean()


def get_submission_predict(df_original: pd.DataFrame, 
                           df_pairs: pd.DataFrame, 
                           labels: np.array,
                           pairs_drop_orders_dublicates = False) -> pd.DataFrame:
    
    # task: понял поздно, уже после написания, но есть проблема, нужно ее решить,
    #       переписать в единственном варианте, либо добавить возможность возвращать
    #       оба варианта, в зависимости от перданного параметра (и проверить на
    #       реальных сабмитах, какой вариант дает больший скор);
    #       проблема: если модель предсказала, что (id1, id2) и (id2, id3) дублиаты,
    #       а (id1, id3) нет, то в текущей реализации в сабмит добавятся строки 
    #       (id1, [id1, id2]), (id2, [id2, id1, id3], (id3, [id3, id2]), хотя на
    #       самом деле они либо все дубликаты, либо где то предсказание ошибочно.
    
    # task: возможно медленно работает, протестировать с прошлой версией во времени
    
    # формируем датасет из пар, для которых match/label == 1
    df_pairs = df_pairs[['id_1', 'id_2']]
    df_pairs['match'] = labels
    df_pairs = df_pairs[df_pairs.match == 1][['id_1','id_2']]
    
    # если мы оставляли пары только в одном направлении (id_1, id_2),
    # то возвращаем что бы они были в обоих (id_1, id_2) и (id_2, id_1)
    if pairs_drop_orders_dublicates:
        df_pairs = (pd.concat([df_pairs, 
                               df_pairs.rename(columns={'id_1': 'id_2', 
                                                        'id_2': 'id_1'})])
                    .drop_duplicates()) #drop_duplicates не обязателен
    
    # добавляем сапоставление  id  самому себе, т.к. этого требует выходной
    # формат
    pairs_one_to_one = pd.DataFrame({'id_1': df_pairs.id_1.unique()})
    pairs_one_to_one['id_2'] = pairs_one_to_one.id_1
    df_pairs = pd.concat([pairs_one_to_one, df_pairs])
    
    # переводим в формат id, matches, где в matches через пробел перечислены все 
    # найденные дубликаты (в том числе сам id попадет в matches)
    df_pairs = (df_pairs.groupby('id_1').id_2.agg(' '.join).to_frame().reset_index()
                .rename(columns={'id_1': 'id', 'id_2': 'matches'}))
    
    # в df_pairs остались только id, для которых найдены дубликаты, мерджим со 
    # всеми id из исходного датасета и добавляем в matchs id самого себя, для 
    # тех id, которые не попали в df_pairs (после merge у них matches == NaN)
    df_submission = pd.merge(df_original['id'], df_pairs, on='id', how='left')
    df_submission['matches'] = df_submission.matches.fillna(df_submission.id)
    
    assert len(df_submission) == len(df_original)
    
    return df_submission


def get_submission_true(df_original: pd.DataFrame) -> pd.DataFrame:
    df_original = df_original[['id', 'point_of_interest']]
    df_poi_matches = (df_original.groupby('point_of_interest').id.agg(' '.join)
                      .to_frame().reset_index().rename(columns={'id': 'matches'}))
    return pd.merge(df_original, df_poi_matches, 
                    on='point_of_interest', how='left')[['id','matches']]

def get_pairs_metrics(df_original: pd.DataFrame, 
                      df_pairs: pd.DataFrame, 
                      labels: np.array,
                      pairs_drop_orders_dublicates = False) -> dict:
    metrics = {}
    submission_true = get_submission_true(df_original)
    submission_pairs_max_true = get_submission_predict(df_original, 
                                                       df_pairs, 
                                                       labels, 
                                                       pairs_drop_orders_dublicates)
    metrics['Jaccard (max)'] = jaccard_score(submission_true, submission_pairs_max_true)
    
    return metrics

def generate_pairs_df(df_dataset: pd.DataFrame, drop_order_dub=False, get_metrics = False, real_test = False) -> (pd.DataFrame, dict):
    # Отбираем только колонки, которые планируем использовать в дальнейшем
    # task: перенести в параметры/гиперпараметры?
    
    FIRST_COLUMN_SELECTION = ['id', 'name', 'latitude', 'longitude', 'country', 'city', 'categories', 'point_of_interest']
    
    # в реальном тесте отсутсвует 'point_of_interest', удаляем из наших колонок
    if real_test:
        FIRST_COLUMN_SELECTION.remove('point_of_interest')
    
    
    df_dataset = df_dataset[FIRST_COLUMN_SELECTION]

    # task: Изучить возможности sklearn для подобных целей, скорей всего это будет наилучший вариант (примерные ключевые слова:
    # sklearn neighbors by coordinate)
    FIRST_COORD_ROUND = 3 # сотые ~= 1 км, тысячные ~= 0.1 км

    # task: Если использовать подобный подход, нужно обязательно производить с перекрытием (придумать как)
    # task: Устранить SettingWithCopyWarning

    # Первоначальный вариант (в 'lat_lon_round' мы получим строкивые представления округленных координат):
    # df_dataset.loc['lat_lon_group'] = (df_dataset.latitude.map(lambda x: str(round(x,FIRST_COORD_ROUND))) + '_' + 
    #                                    df_dataset.longitude.map(lambda x: str(round(x,FIRST_COORD_ROUND))))
    # Альтернативный вариант (результат аналогичный, за исключенем того, что в 'lat_lon_round' мы получим номера групп):
    df_dataset['lat_lon_group'] = df_dataset.groupby([df_dataset.latitude.round(FIRST_COORD_ROUND), 
                                                      df_dataset.longitude.round(FIRST_COORD_ROUND)]).ngroup()
    
    ## Формирование пар-кандидитов
    columns_to_pairs = ['lat_lon_group'] #колонки для совоставления в пары
    df_pairs = pd.merge(df_dataset, df_dataset, on=columns_to_pairs, suffixes=['_1','_2'])

    # Оставляем пары только в одном направлении (id_1, id_2) или в обоих (id_1, id_2) и (id_2, id_1)
    if drop_order_dub:
        df_pairs = df_pairs[df_pairs.id_1 < df_pairs.id_2]
    else: #удаляем только полные дубликаты (id_1, id_1)
        df_pairs = df_pairs[df_pairs.id_1 != df_pairs.id_2]
    
    
    #Generate metrics for current candidates
    metrics = {}
    if get_metrics and not real_test:
        labels = np.array(get_match_label(df_pairs))
        metrics = get_pairs_metrics(df_dataset, 
                                    df_pairs, 
                                    labels,
                                    drop_order_dub)

    return df_pairs, metrics

def add_feauture_geo_distance(df_pairs: pd.DataFrame, normalize=False, prefix='ftr_') -> pd.DataFrame:
    # Считаем расстояние в км между точками
    # task: Возможно нет смысла считать точно через haversine (с учетом шарообразности земли), 
    # можно считать более грубо, но быстрее
    
    df_pairs[f'{prefix}geo_distance'] = df_pairs.apply(lambda x: hs.haversine((x.latitude_1,x.longitude_1), 
                                                 (x.latitude_2,x.longitude_2), 
                                                 unit=Unit.KILOMETERS), axis=1)
    return df_pairs


def add_feauture_levenshtein_distance(df_pairs: pd.DataFrame, normalize=False, prefix='ftr_') -> pd.DataFrame:
    # Levenshtein distance of names
    df_pairs[f'{prefix}name_levenshtein'] = df_pairs.apply(lambda x: fuzz.token_set_ratio(x.name_1,
                                                                                          x.name_2), axis=1)
    return df_pairs


def run_futures_pipeline(dataset: pd.DataFrame, futures_pipeline: list, prefix='ftr_') -> pd.DataFrame:
    for step in futures_pipeline:
        dataset = step['func'](dataset, prefix=prefix, **step['params'])
    future_columns = [col for col in dataset.columns if col.startswith(prefix)]
    return dataset.reset_index(drop=True), future_columns


def get_match_label(dataset: pd.DataFrame) -> pd.Series: # 1 = match, 0 = not match
    #Наша целевая переменная (label/target)
    return (dataset['point_of_interest_1'] == dataset['point_of_interest_2']).astype(int)


In [16]:
os.path.join('.',"data", "raw")

In [12]:
os.path.join('.',"output","log.txt")

In [17]:
PATH_TO_RAW_DATA = os.path.join('.',"data", "raw")

In [18]:
os.path.join(PATH_TO_RAW_DATA, 'train.csv')

In [ ]:
## Hyperparameters

# Задаем пути до директории с train/test.csv (в записимости от варианта запуска ноутбука)
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    PATH_TO_RAW_DATA = '/kaggle/input/foursquare-location-matching'
else:
    #./data/raw
    PATH_TO_RAW_DATA = os.path.join('.',"data", "raw")
    
# Количество частей, на которые будет разбит train датасет, 
# одна часть уйдет на split_test, остальные на split_train
# 3 = 1/2 (или 33%/88%), 4 = 1/3 (или 25%/75) и т.д.
TRAIN_TEST_N_SPLITS = 3

# Перезапуск всего обучения на полном train.csv и формирование/сохранение submission.csv
# на основе test.csv, необходимо для реального сабмита на каггле
GENERATE_SUBMISSION_CSV = True

## Обработка датасетов

### - Разбиваем исходный Raw Train датасет (train.csv) на Train/Test (-> имеем на выходе split_train/split_test)

In [ ]:
# Разбиваем так, что бы объекты с одинаковым POI не раскидывались в разные выборки

train = pd.read_csv(os.path.join(PATH_TO_RAW_DATA, 'train.csv'))

# Подход по применению GroupKFold для подобных целей подсмотрел здесь:
# https://www.kaggle.com/code/ryotayoshinobu/foursquare-lightgbm-baseline

kf = GroupKFold(n_splits=TRAIN_TEST_N_SPLITS)
for i, (trn_idx, val_idx) in enumerate(kf.split(train, train['point_of_interest'], train['point_of_interest'])):
    train.loc[val_idx, "parts"] = str(i)
    
split_test = train[train.parts == '1'].drop(columns='parts')
split_train = train[~(train.parts == '1')].drop(columns='parts')

print(f'Our train size: {len(split_train)}, Our test size: {len(split_test)}')

#Освобождаем память
del train

### - Предобработка датасета, отбор, формирование пар-кандидатов для сравнения (-> датасет пар-кандидатов)

In [ ]:
PAIRS_DROP_ORDER_DUBLICATES = True

pairs_train, pairs_metrics = generate_pairs_df(split_train, PAIRS_DROP_ORDER_DUBLICATES, get_metrics=True)
print(pairs_metrics)
pairs_train

### - Фиче инжинеринг датасета пар-кандидатов, формирование X, y (-> X, y, готовые для передачи в модель)

In [ ]:
## Формирование датасета, подходящего для передачи в модель
pairs_futures_pipeline = [
    {'func': add_feauture_geo_distance,
     'params': {}},
    {'func': add_feauture_levenshtein_distance,
     'params': {}}]

#Генерируем все необходимые фичи
pairs_train, future_columns = run_futures_pipeline(pairs_train, pairs_futures_pipeline)

In [ ]:
## Нормализация фичей (или нужно сразу при формировании фичей?)

In [ ]:
## Формируем X, y для дальнейшей передачи в модель

y = get_match_label(pairs_train)
X = pairs_train[future_columns]

# оставляем в памяти (для очистки лишней памяти) только те, колонки, 
# которые нам понадобятся в дальнейшем (для формирования submission)
pairs_train = pairs_train[['id_1', 'id_2']]

### - Обработка датасета отложенной выборки split_test для итоговой оценки модели (-> )

In [ ]:
pairs_test, pairs_metrics = generate_pairs_df(split_test, PAIRS_DROP_ORDER_DUBLICATES, get_metrics=True)

print(pairs_metrics)

pairs_test, future_columns = run_futures_pipeline(pairs_test, pairs_futures_pipeline)

y_test = get_match_label(pairs_test)
X_test = pairs_test[future_columns]

# оставляем в памяти (для очистки лишней памяти) только те, колонки, 
# которые нам понадобятся в дальнейшем (для формирования submission)
pairs_test = pairs_test[['id_1', 'id_2']]

## Обучение модели (-> сохраненная в файл модель)

In [ ]:
# Model params
model_params = {'random_state': 42,
                'n_estimators': 10,
                'verbosity': 0}

# Define the model
model = XGBClassifier(**model_params)

# Fit the model
model.fit(X, y)

## Оценка модели на отложенной выборке (-> Score на отложенной выборке)

In [ ]:
# Get predictions
y_pred = model.predict(X_test)

# Accuracy (not all id's) удалить, т.к. не учитывает часть id
print(np.mean(y_pred == np.array(y_test)))

In [ ]:
submission_true = get_submission_true(split_test)
submission_pred = get_submission_predict(split_test, pairs_test, y_pred, PAIRS_DROP_ORDER_DUBLICATES)

In [ ]:
jaccard_score(submission_true, submission_pred)

## Перезапуск всего пайплайна на полном датасете для целей сабмита на каггле (-> submission.csv для сабмита на каггле)

In [ ]:
if GENERATE_SUBMISSION_CSV:
    train = pd.read_csv(os.path.join(PATH_TO_RAW_DATA, 'train.csv'))

    
    ## Отбираем кандидатов и формируем парный датасет
    pairs_train, pairs_metrics = generate_pairs_df(train, PAIRS_DROP_ORDER_DUBLICATES, get_metrics=True)
    print(pairs_metrics)

    
    ## Генерируем все необходимые фичи
    pairs_train, future_columns = run_futures_pipeline(pairs_train, pairs_futures_pipeline)
    
    
    ## Формируем X, y для дальнейшей передачи в модель
    y = get_match_label(pairs_train)
    X = pairs_train[future_columns]

    # оставляем в памяти (для очистки лишней памяти) только те, колонки, 
    # которые нам понадобятся в дальнейшем (для формирования submission)
    pairs_train = pairs_train[['id_1', 'id_2']]
    
    
    ## Обучаем модель
    
    # Model params
    model_params = {'random_state': 42,
                    'n_estimators': 10,
                    'verbosity': 0}

    # Define the model
    model = XGBClassifier(**model_params)

    # Fit the model
    model.fit(X, y)
    
    
    ## Генерируем предсказания и финальный submission.csv
    test = pd.read_csv(os.path.join(PATH_TO_RAW_DATA, 'test.csv')
    
    # Т.к. публичный (в отличе от приветного) test.csv содержит всех 5 разрозненных записей, которые даже не
    # попадают в кандидаты, весь пайплайн рушится, по причине пустого парного датасета,
    # что бы это обойти, проверяем на количество строк (с целью определить имеем мы дело с публичным или 
    # приватным test.csv) и в случае публичного, дублируем объекты изменив предварительно id
    if len(test) == 5:
        temp = test.copy()
        temp.id = temp.id + '_'
        test = pd.concat([test, temp])
    
    pairs_test, _ = generate_pairs_df(test, PAIRS_DROP_ORDER_DUBLICATES, real_test=True)

    pairs_test, future_columns = run_futures_pipeline(pairs_test, pairs_futures_pipeline)

    X_test = pairs_test[future_columns]

    # оставляем в памяти (для очистки лишней памяти) только те, колонки, 
    # которые нам понадобятся в дальнейшем (для формирования submission)
    pairs_test = pairs_test[['id_1', 'id_2']]
    
    # Get predictions
    y_pred = model.predict(X_test)

    submission_pred = get_submission_predict(test, pairs_test, y_pred, PAIRS_DROP_ORDER_DUBLICATES)
    
    submission_pred.to_csv("submission.csv", index=False)